# 2.1 Dataset Artigos SciELO

## Imports

In [5]:
import os

import pandas as pd
import numpy as np
from bs4 import BeautifulSoup, SoupStrainer
from tqdm.notebook import tqdm

## Código

In [6]:
path_artigos = os.path.join(os.getcwd(), 'artigos_brutos', 'SciELO')

In [7]:
# contagem de artigos
soma = sum(len(os.listdir(os.path.join(path_artigos, a))) for a in os.listdir(path_artigos))
print(f'Total de artigos: {soma}')

Total de artigos: 1681


### Criando o DataSet

In [22]:
dict_df = {'title': [], 'seção': [], 'ano': [],  'doi_id': [], 'text_html': []}

for ano in tqdm(os.listdir(path_artigos)):
    path_abs  = os.path.join(path_artigos, ano)
    
    for arquivo in os.listdir(path_abs):
        arq_path = os.path.join(path_abs, arquivo)
        
        with open(arq_path, 'r', encoding='utf8') as arq:
            texto_html = arq.read()
            
        soup = BeautifulSoup(texto_html, 'lxml')
        
        try:
            doi_id = soup.find('a', attrs={'class': '_doi'}).text.strip().removeprefix('https://doi.org/')
        except AttributeError:
            doi_id = None

        try:
            titulo = ' '.join(soup.find('h1', attrs={'class': 'article-title'}).text.split())
        except AttributeError:
            titulo = None

        try:
            seção = soup.find('span', attrs={'class': '_articleBadge'}).text.strip().lower()
        except AttributeError:
            seção = None

        dict_df['title'].append(titulo)
        dict_df['doi_id'].append(doi_id)
        dict_df['text_html'].append(texto_html)
        dict_df['seção'].append(seção)
        dict_df['ano'].append(int(ano))
        

  0%|          | 0/22 [00:00<?, ?it/s]

In [23]:
# salvando o dataset em feather
df = pd.DataFrame(dict_df)
df['ano'] = df['ano'].astype(np.uint16)
df.to_feather('scielo_dataset.feather')

In [24]:
df.head()

,title,seção,ano,doi_id,text_html
0,"""... E o XIV SNEF aconteceu""",editorial,2001,10.1590/S1806-11172001000200001,"<!DOCTYPE html>\n<div class=""articleTxt"">\n <d..."
1,Carta ao editor,carta ao editor,2001,10.1590/S1806-11172001000200002,"<!DOCTYPE html>\n<div class=""articleTxt"">\n <d..."
2,Pontos quânticos: átomos artificiais e transis...,,2001,10.1590/S1806-11172001000200004,"<!DOCTYPE html>\n<div class=""articleTxt"">\n <d..."
3,As equações de Hamilton sem transformação de L...,,2001,10.1590/S1806-11172001000200005,"<!DOCTYPE html>\n<div class=""articleTxt"">\n <d..."
4,Um experimento de oscilador forçado amortecido,,2001,10.1590/S1806-11172001000200006,"<!DOCTYPE html>\n<div class=""articleTxt"">\n <d..."
